In [1]:
import uvage
from random import randrange, randint
import os.path

#GLobal Variables ----------------------------------------------------------------
score = 0
game_on = False
game_over = False
game_over_screen = False
player_name = input('Name:')
scoreboard_values = []
tick_rate = 20
multiplier = 1

#Set-up Game Objects ----------------------------------------------------------------
camera = uvage.Camera(600, 600)
scoreboard = uvage.from_color(300, 300, 'white', 400, 400)
button = uvage.from_color(300, 500, 'red', 400, 20)
score_table = []
current_frame = 0

coin = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), r'Snake Sprite Sheets\coin.png')

freeze_image = uvage.load_sprite_sheet(r'Snake Sprite Sheets\freeze_float.png', 3, 3)
freeze = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), freeze_image[-1])
freeze.size = [50, 50]
freeze_spawn = False
freeze_on = False

double_score = uvage.load_sprite_sheet(r'Snake Sprite Sheets\2xscore_float.png', 3, 3)
double = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), double_score[-1])
double.size = [50, 50]
multiplier_spawn = False
multiplier_on = False

apple_image = uvage.load_sprite_sheet(r'Snake Sprite Sheets\apple_float.png', 3, 3)
apple = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), apple_image[0])
apple.size = [50, 50]

snake_headsheet = uvage.load_sprite_sheet(r'Snake Sprite Sheets\snake eat.png', 2, 2)

snake_head = uvage.from_image(300, 300, snake_headsheet[0])
snake_head.size = [30, 30]

direction = 'RIGHT'
change_to = direction

snake_body = uvage.load_sprite_sheet(r'Snake Sprite Sheets\snake_body.png', 1, 1) #loads the sprite sheet

#Start the Snake w/ 4 Body Pieces + 1 Head
snake_list = [[300, 300],
              [290, 300],
              [280, 300],
              [270, 300]
              ]



#Inter-Session Progress ----------------------------------------------------------------

def store_highscore():

    global player_name, score

    #Checks if there is a highscore.txt file if not makes one and writes to it
    if os.path.isfile('highscore.txt') == False:
        file = open('highscore.txt', 'x')
        file.write(player_name + ',' + str(score))
        file.close()
    #Writes to highscore.txt 
    else:
        highscore_file = open('highscore.txt', 'a')
        highscore_file.write('\n' + player_name + ',' + str(score))
        highscore_file.close()

    return

def clear_highscore():
    #Clears highscore.txt
    highscore_file = open('highscore.txt', 'w')
    print(file = highscore_file)
    highscore_file.close()

    return

def retrieve_highscorelist():

    #Reads the file and sorts it
    highscore_file = open('highscore.txt', 'r')
    highscore = []
    
    for row in highscore_file.read().split('\n'):
        highscore.append(row.split(','))

    highscore_file.close()

    highscore.sort(key = lambda x: float(x[1]), reverse = True)
    
    return highscore 

def create_score_table(values):

    #Creates the score_table to be shown at the end
    global score_table
    
    scoreboard_values = values
    score_table = []

    rank = 0
    entries = 1 

    while entries < 11 and entries <= (len(scoreboard_values)):
        text = "{} | Score:{}".format(scoreboard_values[rank][0], scoreboard_values[rank][1])
        entry_diplay = uvage.from_text(scoreboard.x, scoreboard.top - 20 + 40 * entries, text, 40, "black")
        score_table.append(entry_diplay)
        rank += 1
        entries += 1

    return

# Collectibles -------------------------------

def handle_coin():
    global score, coin

    camera.draw(coin)

    if coin.touches(snake_head):
        coin = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), r'Snake Sprite Sheets\coin.png')
        score += 100 * multiplier

    return

def handle_freeze():
    global score, tick_rate, freeze, freeze_image, freeze_on, freeze_spawn


    if freeze_spawn == False:
        if randint(0, 100 * tick_rate) < tick_rate:
            freeze_spawn = True

    if freeze_spawn == True:
        freeze = uvage.from_image(freeze.x, freeze.y, freeze_image[current_frame%6])
        freeze.size = [50, 50]
        camera.draw(freeze)

    if freeze_on == True:
        camera.draw("2x Freeze On", 36, "white", 300, 100)
        if current_frame%100 == 0:
            freeze_on = False

    if snake_head.touches(freeze):
        freeze = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), freeze_image[0])
        freeze_on = True
        freeze_spawn = False

def handle_double():
    global score, double, multiplier, multiplier_on, multiplier_spawn

    if multiplier_spawn == False:
        if randint(0, 100 * tick_rate) < tick_rate:
            multiplier_spawn = True

    if multiplier_spawn == True:
        double = uvage.from_image(double.x, double.y, double_score[current_frame%6])
        double.size = [50, 50]
        camera.draw(double)



    if multiplier_on == True:
        camera.draw("Multiplier On", 36, "white", 300, 150)

        if current_frame%100 == 0:
            multiplier_on = False
            multiplier = 1


    if snake_head.touches(double):
        double = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), double_score[0])

        multiplier = 2
        multiplier_on = True
        multiplier_spawn = False


def handle_apple():
    global score, apple, apple_image, current_frame, snake_list, snake_body, snake_head

    apple = uvage.from_image(apple.x, apple.y, apple_image[current_frame%9])
    apple.size = [50, 50]
    camera.draw(apple)

    if apple.touches(snake_head):
        apple = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), apple_image[0])
        apple.size = [50, 50]
        score += 100 * multiplier
        #Adds another position tracker  to the snake
        snake_list.append(list((snake_head.x, snake_head.y)))
    else:
        snake_list.pop()


    return

def score_display():
	global score

	camera.draw("Score: " + str(int(score)), 36, "white", 300, 50) #displays the score while they're playing the game

# Snake --------------------------------

def handle_snake():
    global facing_right, facing_up, direction, game_over, change_to, snake_list, snake_head, freeze_on

    if game_over == True:
        snake_head.xspeed = 0
        snake_head.yspeed = 0

    if game_on == True:

        if snake_head.x < 0 or snake_head.x > 600 or snake_head.y < 0 or snake_head.y > 600:
            game_over = True
            store_highscore()

        if uvage.is_pressing("up arrow"):
            change_to = 'UP'
                
        elif uvage.is_pressing("down arrow"):
            change_to = 'DOWN'

        elif uvage.is_pressing("left arrow"):
            change_to = 'LEFT'

        elif uvage.is_pressing("right arrow"):
            change_to = 'RIGHT'

        if change_to == 'UP' and direction != 'DOWN':
            direction = 'UP'
            snake_head = uvage.from_image(snake_head.x, snake_head.y, r'Snake Sprite Sheets\snake eat up.png')
        if change_to == 'DOWN' and direction != 'UP':
            direction = 'DOWN'
            snake_head = uvage.from_image(snake_head.x, snake_head.y, r'Snake Sprite Sheets\snake eat down.png')
        if change_to == 'LEFT' and direction != 'RIGHT':
            direction = 'LEFT'
            snake_head = uvage.from_image(snake_head.x, snake_head.y, r'Snake Sprite Sheets\snake eat left.png')
        if change_to == 'RIGHT' and direction != 'LEFT':
            direction = 'RIGHT'
            snake_head = uvage.from_image(snake_head.x, snake_head.y, r'Snake Sprite Sheets\snake eat right.png')

        
        if freeze_on == True:
            if direction == 'UP':
                snake_head.y -= 15
            if direction == 'DOWN':
                snake_head.y += 15
            if direction == 'LEFT':
                snake_head.x -= 15
            if direction == 'RIGHT':
                snake_head.x += 15

        else:
            if direction == 'UP':
                snake_head.y -= 20
            if direction == 'DOWN':
                snake_head.y += 20
            if direction == 'LEFT':
                snake_head.x -= 20
            if direction == 'RIGHT':
                snake_head.x += 20
        
        snake_head.size = [30,30]
        camera.draw(snake_head)

        #Constantly Updates the Location of the Snake with Insert & Pop
        snake_list.insert(0, list((snake_head.x, snake_head.y)))

        for position in snake_list[1:]:
            body = uvage.from_image(position[0], position[1], r'Snake Sprite Sheets\snake_body.png')
            body.size = [50,50]
                
            camera.draw(body)
    
        for position in snake_list[1:]:
            if snake_head.contains(position):
                game_over = True
                store_highscore()

    
    return


# Restart -----------------------------------------
def handle_gameover():
    global game_over, game_on, game_over_screen, scoreboard_values

    scoreboard_values = retrieve_highscorelist()
    create_score_table(scoreboard_values)
    game_on = False
    
    return

def handle_gameover_screen():
    global game_over, game_on, game_over_screen, score_table

    camera.draw(scoreboard)
    camera.draw(button)
    camera.draw("Close", 36, "white", 300, 500)
    for row in score_table:
        camera.draw(row)
    if button.contains(camera.mouse) and camera.mouseclick == True:
        camera.clear('light blue')
        restart_game()
        game_over = False
    

    return

def restart_game():
    global score, snake_list, snake_head, direction, change_to

    score = 0

    snake_head.x = 300
    snake_head.y = 300

    direction = 'RIGHT'
    change_to = direction

    snake_list = [[300, 300],
                [290, 300],
                [280, 300],
                [270, 300]
                ]

    return

# Game Code ----------------------------------------------------------------

def game():
    global snake, current_frame, game_on

    camera.clear('light blue')

    if game_on == False:
        camera.draw("Hit Space to Start!", 100, "white", 300, 300)

    if uvage.is_pressing('space'):
        game_on = True

    if game_on == True:
        score_display()
        handle_snake()
        handle_apple()
        handle_double()
        handle_freeze()
        handle_coin()
        current_frame += 1

    if game_over == True:
        handle_gameover()
        handle_gameover_screen()

    camera.display()
    return

#Run Game ----------------------------------------------------------------
uvage.timer_loop(tick_rate, game)
#Finished

pygame 2.1.2 (SDL 2.0.18, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
